In [1]:
%%capture

# https://www.kaggle.com/bguberfain/openai-clip-with-train/notebook

import sys
!cp -r ../input/openai-clip/CLIP/CLIP-main /tmp/

# Kaggle likes to unpack .gz files in datasets... so we have to pack it back
!gzip -c /tmp/CLIP-main/clip/bpe_simple_vocab_16e6.txt > /tmp/CLIP-main/clip/bpe_simple_vocab_16e6.txt.gz
sys.path.append('/tmp/CLIP-main')

!pip install ../input/openai-clip/ftfy-5.9/ftfy-5.9
!pip install ../input/openai-clip/torch-1.7.1+cu110-cp37-cp37m-linux_x86_64.whl \
             ../input/openai-clip/torchvision-0.8.2+cu110-cp37-cp37m-linux_x86_64.whl \
             ../input/faiss-163/faiss_gpu-1.6.3-cp37-cp37m-manylinux2010_x86_64.whl

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from os.path import join
import numpy as np
import pandas as pd
import clip, os, skimage
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm

clip.available_models()

In [3]:
model, preprocess = clip.load("../input/openai-clip/ViT-B-32.pt", jit=False)
model = model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

In [4]:
train_image_path = Path("../input/petfinder-pawpularity-score/train")
file_names = [f.name for f in train_image_path.iterdir() if f.suffix == ".jpg"]

In [5]:
original_images = []
images = []
plt.figure(figsize=(15, 12))

for filename in file_names[:9]:
    image = Image.open(join(train_image_path, filename))
  
    plt.subplot(3, 3, len(images) + 1)
    plt.imshow(image)
    plt.xticks([])
    plt.yticks([])

    original_images.append(image)
    images.append(preprocess(image))

In [6]:
texts = ['Cute',
         'Funny',
         'Derp', # let's see if this works
         'Small',
         'Happy',
         'Sad',
         'Aggressive',
         'Friendly',
         'Old',
         'Young',
         'Love']

In [7]:
image_input = torch.tensor(np.stack(images)).cuda()
text_tokens = clip.tokenize(texts).cuda()

# text_tokens = clip.tokenize([f"A {w} photo of a" + w for w in texts]).cuda()

In [8]:
with torch.no_grad():
    image_features = model.encode_image(image_input).float()
    text_features = model.encode_text(text_tokens).float()

image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)

similarity_matrix = torch.inner(text_features, image_features).cpu()

In [9]:
count = len(texts)

plt.figure(figsize=(20, 16))
plt.imshow(similarity_matrix, vmin=0.1, vmax=0.3, cmap = 'RdBu')

plt.yticks(range(count), texts, fontsize=18)
plt.xticks([])

for i, image in enumerate(original_images):
    plt.imshow(image, extent=(i - 0.5, i + 0.5, -1.6, -0.6), origin="lower")
for x in range(similarity_matrix.shape[1]):
    for y in range(similarity_matrix.shape[0]):
        plt.text(x, y, f"{similarity_matrix[y, x]:.2f}", ha="center", va="center", size=12)

for side in ["left", "top", "right", "bottom"]:
    plt.gca().spines[side].set_visible(False)

plt.xlim([-0.5, count - 0.5])
plt.ylim([count + 0.5, -2])

plt.title("Cosine similarity matrix between text and image features", size=20, loc='left')
plt.show()

In [10]:
class PetDataset(Dataset):
    def __init__(self, path):
        self.path = path
        self.files = [f for f in path.iterdir() if f.suffix == ".jpg"]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        _im_path = self.files[idx]
        _img = Image.open(_im_path)
        _img = preprocess(_img)
        return _img, _im_path.name.split('.')[0]

In [11]:
def create_similarity_features(dl):
    features = []
    names = []
    with torch.no_grad():
        for xb, name in dl:
            xb = xb.cuda()
            xb = model.encode_image(xb)
            xb /= xb.norm(dim=-1, keepdim=True)
            sim_matrix = torch.inner(text_features, xb.float()).cpu().numpy()
            features.append(sim_matrix)
            names.append(name)
    return features, names

In [12]:
ds = PetDataset(train_image_path)
dl = DataLoader(ds, batch_size = 400, shuffle=False)
train_features, train_names = create_similarity_features(dl)

In [13]:
train_features_df = pd.DataFrame(np.hstack(train_features).T,
                           index = np.hstack(train_names).T,
                           columns = texts)
df_corr = train_features_df.corr()
plt.figure(figsize=(13,8))

plt.title("Correlation matrix between engineered features", size=20, loc='left') 
sns.heatmap(df_corr, cmap='RdBu', annot=True, linewidths=2)

In [14]:
# saving training features for later use
train_features_df.to_csv('clip_features.csv')

---

In [19]:
!python3 -m pip install scikit-garden

In [20]:
import os
import cv2
import time
import json
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt


from PIL import Image
from enum import Enum
from glob import glob
from tqdm import trange
from pprint import pprint
from colorama import Fore
from tqdm import tqdm_notebook
from sklearn.utils import shuffle
from IPython.display import display


from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.model_selection import *



# Neural Network Models
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import Input, Concatenate, BatchNormalization
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, Dropout



# Regression Models
import six
import sys
sys.modules['sklearn.externals.six'] = six
from skgarden import RandomForestQuantileRegressor,MondrianForestRegressor,MondrianTreeRegressor
from sklearn.linear_model import Lasso
#from skgarden import RandomForestQuantileRegressor
from statsmodels.formula.api import quantreg
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor,VotingRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from skgarden.quantile import ExtraTreesQuantileRegressor,DecisionTreeQuantileRegressor

warnings.filterwarnings("ignore")
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [21]:
train_df = pd.read_csv("../input/same-old-creating-folds/train_10folds.csv")
test_df = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
sample_submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
test_image_path = train_image_path = Path("../input/petfinder-pawpularity-score/test")

useful_features = [
    'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
    'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur',
    'Cute', 'Funny', 'Derp', 'Small', 'Happy', 'Sad', 'Aggressive',
    'Friendly', 'Old', 'Young', 'Love']

In [22]:
ds = PetDataset(test_image_path)
dl = DataLoader(ds, batch_size = 400, shuffle=False)
test_features, test_names = create_similarity_features(dl)

test_features_df = pd.DataFrame(np.hstack(test_features).T,
                                index = np.hstack(test_names).T,
                                columns = texts)

In [23]:
train_df = train_df.join(train_features_df, on = 'Id')
test_df = test_df.join(test_features_df, on = 'Id')

In [24]:
Result_set = {
    "Model" : [],
    "rmse_train" : [],
    "rmse_valid" : []
}

In [42]:
def generate_predictions( fold, df, df_test, useful_features):    
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.Pawpularity
    yvalid = xvalid.Pawpularity

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    xtest = xtest[useful_features]
    
    
    Model_dict = {
    "Lasso":Lasso(alpha = 0.1),
    "SVR": SVR(C=1.0, epsilon=0.2, kernel ='rbf'),
    "Ridge": Ridge(alpha = 1.0),
    "KernelRidge":KernelRidge(alpha=0.1),
    "ElasticNet": ElasticNet(random_state=0),
    "XGBRegressor": XGBRegressor(n_estimators=1000, max_depth=5, eta=0.01, subsample=0.7, colsample_bytree=0.6),
    "LGBMRegressor": LGBMRegressor(),
    "LinearRegression": LinearRegression(),
    "BaggingRegressor": BaggingRegressor(base_estimator=CatBoostRegressor(), n_estimators=10, random_state=0),
    "ExtraTreesRegressor": ExtraTreesRegressor(n_estimators=100, random_state=0),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(max_depth=2, random_state=0),
    "GaussianProcessRegressor": GaussianProcessRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    "AdaBoostRegressor": AdaBoostRegressor(n_estimators=100),
    "KNeighborsRegressor": KNeighborsRegressor(n_neighbors=5),
    "CatBoostRegressor": CatBoostRegressor(iterations=200, learning_rate=0.01),
    "random forest quantile regression" : RandomForestQuantileRegressor(random_state=0, min_samples_split=10, n_estimators=1000),
    "mondrian tree" : MondrianTreeRegressor(random_state=1, max_depth=2),
    "Mondrian Forest Regression":MondrianForestRegressor(random_state=1, max_depth=2),
    "extra trees qunatile regressor":ExtraTreesQuantileRegressor(n_estimators=100, random_state=0),
    "Decision tree Quantile regressor":DecisionTreeQuantileRegressor(random_state=0),
    
    }
    
    cbr = CatBoostRegressor(iterations=200, learning_rate=0.01)
    gbr = GradientBoostingRegressor()
    xgbr = XGBRegressor()
    
    Model_list = list(Model_dict.keys())

    for i in tqdm_notebook(range(len(Model_list))):
        model_name = Model_list[i]
        model = Model_dict[model_name]
    
       
        
        model.fit(xtrain, ytrain)
        preds_valid = model.predict(xvalid)
        test_preds = model.predict(xtest)
        rmse = mean_squared_error(yvalid, preds_valid, squared=False)
        print(rmse)

        
    


    
    return test_preds

In [45]:
final_predictions = []
rmse_l=[]
for fold_ in range(10):
    final_predictions.append(
        generate_predictions(
            
            fold=fold_,
            df=train_df,
            df_test=test_df,
            useful_features=useful_features,
            #print("Fold", fold_)
        )
    )


In [46]:
final_predictions = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.Pawpularity = final_predictions
sample_submission.to_csv("submission.csv", index=False)
